In [2]:
import json
import os
import csv
from time import sleep

import nibabel as nib
import numpy as np
import pydicom as dcm
from itkwidgets import view

In [2]:
def get_metadata(filename):
    cases = {}
    with open(filename, 'r') as csvf:
        csv_reader = csv.reader(csvf)
        header = next(csv_reader)
        for row in csv_reader:
            if not row[4].startswith("MED"):
                continue
            cases[row[4]] = {header[i]:row[i] for i in range(len(row))}
    return cases

def read_dicom(case: dict):
    filename = "Dataset" + case['File Location'][1:].replace('\\', '/')
    dcms = os.listdir(filename)
    dcms.sort()
    first_image = dcm.read_file(f"{filename}/{dcms[0]}")
    first_pixs = first_image.pixel_array
    volume = np.empty((first_pixs.shape[0], first_pixs.shape[1], len(dcms)))
    for idx, im in enumerate(dcms):
        pixels = dcm.read_file(f"{filename}/{im}").pixel_array
        volume[:,:,idx] = pixels.transpose()
    return volume

def read_mask(case: dict):
    case_name = case['File Location'][1:].split('\\')[2]
    filename = "Dataset/MED_ABD_LYMPH_MASKS/" + case_name + "/" + case_name + "_mask.nii.gz"
    mask = nib.load(filename).get_fdata()
    return mask

In [3]:
cases = get_metadata("Dataset/metadata.csv")

In [4]:
with open("bp_and_cubes_data_64.json", "r") as f:
    prism_data = json.loads(f.read())

### Find the min bounding prism shape for all axes across all data.

In [5]:
min_shape = [1e9, 1e9, 1e9]

for case in prism_data.values():
    bp_shape = case["bp_shape"]

    for axis in range(3):
        axis_shape = bp_shape[axis]
        min_shape[axis] = axis_shape if axis_shape < min_shape[axis] else min_shape[axis]
    
print(f"{min_shape = }")

min_shape = [300, 192, 175]


In [6]:
prob_map = np.zeros(min_shape)

for case_num, case in enumerate(cases.keys()):
    bp_shape = prism_data[case]["bp_shape"]
    bp_coords = prism_data[case]["bp_extents"]

    mask = read_mask(cases[case])

    index_okay = True
    for coords, voxel in np.ndenumerate(mask[bp_coords[0][0]:bp_coords[0][1],
                                             bp_coords[1][0]:bp_coords[1][1],
                                             bp_coords[2][0]:bp_coords[2][1]]):
        coords = list(coords)
        for axis in range(3):
            coords[axis] += bp_coords[axis][0]
        if not voxel:
            continue
        try:
            prob_map[int((coords[0]-bp_coords[0][0])*(min_shape[0]-1) / (bp_shape[0]-1)),
                    int((coords[1]-bp_coords[1][0])*(min_shape[1]-1) / (bp_shape[1]-1)),
                    int((coords[2]-bp_coords[2][0])*(min_shape[2]-1) / (bp_shape[2]-1))] += 1
        except IndexError:
            if index_okay:
                print((f"{case = }"
                       f"{bp_shape = }"
                       f"{bp_coords = }"))
                index_okay = False
            continue
    print(f"Progress: {(case_num+1)/len(cases.keys())*100:2.2f}%.", end='\r')
    sleep(0)

    with open("prob_map.npy", "wb") as f:
        np.save(f, prob_map)

    with open("prob_map_progress.txt", "w+") as f:
        f.write(f"Done with {case_num+1} of {len(cases.keys())}.")

In [3]:
with open("prob_map.npy", "rb") as f:
    read_prob_map = np.load(f)

In [4]:
view(read_prob_map)

Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itk.itkImagePython.itkImageD3; pro…